In [ ]:
## 3-1 softmax only -- training time 6.95s, accuracy 82.70% ##

In [31]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5


def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [32]:
## restore HW2 model ##
restore_saver = tf.train.import_meta_graph("Team58_HW2\Team58_HW2.ckpt.meta")

## get tensor from HW2 ##
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

In [33]:
## setting trainable variable ##
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(0.01, name="Adamgram")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
HW3_1_saver = tf.train.Saver() # create saver

In [34]:
import time

## parameter setting ##
n_epochs = 1000
early_stop_max_steps = 20
steps_without_progress = 0
best_loss = 9999999

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "Team58_HW2\Team58_HW2.ckpt")
    for var in output_layer_vars:
        var.initializer.run()

        
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_train2, y: y_train2})
        now_loss = sess.run(loss, feed_dict={X: X_valid2, y: y_valid2})
        now_acc = sess.run(accuracy, feed_dict={X: X_valid2, y: y_valid2})
        
        ## early stop implementation ##
        if now_loss < best_loss:
            HW3_1_saver.save(sess, "Team58_HW3_1\Team58_HW3_1.ckpt")
            best_loss = now_loss
            steps_without_progress = 0
        else:
            steps_without_progress += 1
            if steps_without_progress > early_stop_max_steps: ## if no progress in 20 step then early stop ##
                print("early stop")
                break 
        ## print result per epoch ##
        print("{}\tValidation loss: {:.4f}\tBest loss: {:.4f}\tAccuracy: {:.2f}%".format(epoch, now_loss, best_loss, now_acc * 100))

    HW3_1_saver.restore(sess, "Team58_HW3_1\Team58_HW3_1.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("test accuracy",acc_test * 100,"%")

INFO:tensorflow:Restoring parameters from Team58_HW2\Team58_HW2.ckpt
0	Validation loss: 1.5964	Best loss: 1.5964	Accuracy: 26.67%
1	Validation loss: 1.5050	Best loss: 1.5050	Accuracy: 40.00%
2	Validation loss: 1.4554	Best loss: 1.4554	Accuracy: 44.00%
3	Validation loss: 1.4194	Best loss: 1.4194	Accuracy: 49.33%
4	Validation loss: 1.3876	Best loss: 1.3876	Accuracy: 50.67%
5	Validation loss: 1.3605	Best loss: 1.3605	Accuracy: 54.67%
6	Validation loss: 1.3437	Best loss: 1.3437	Accuracy: 56.00%
7	Validation loss: 1.3365	Best loss: 1.3365	Accuracy: 56.67%
8	Validation loss: 1.3361	Best loss: 1.3361	Accuracy: 57.33%
9	Validation loss: 1.3348	Best loss: 1.3348	Accuracy: 58.00%
10	Validation loss: 1.3298	Best loss: 1.3298	Accuracy: 57.33%
11	Validation loss: 1.3221	Best loss: 1.3221	Accuracy: 59.33%
12	Validation loss: 1.3148	Best loss: 1.3148	Accuracy: 59.33%
13	Validation loss: 1.3091	Best loss: 1.3091	Accuracy: 58.67%
14	Validation loss: 1.3037	Best loss: 1.3037	Accuracy: 58.00%
15	Validati

135	Validation loss: 1.0662	Best loss: 1.0662	Accuracy: 85.33%
136	Validation loss: 1.0662	Best loss: 1.0662	Accuracy: 84.67%
137	Validation loss: 1.0663	Best loss: 1.0662	Accuracy: 84.67%
138	Validation loss: 1.0667	Best loss: 1.0662	Accuracy: 84.67%
139	Validation loss: 1.0672	Best loss: 1.0662	Accuracy: 84.67%
140	Validation loss: 1.0677	Best loss: 1.0662	Accuracy: 84.67%
141	Validation loss: 1.0681	Best loss: 1.0662	Accuracy: 85.33%
142	Validation loss: 1.0684	Best loss: 1.0662	Accuracy: 85.33%
143	Validation loss: 1.0684	Best loss: 1.0662	Accuracy: 85.33%
144	Validation loss: 1.0682	Best loss: 1.0662	Accuracy: 85.33%
145	Validation loss: 1.0678	Best loss: 1.0662	Accuracy: 84.67%
146	Validation loss: 1.0674	Best loss: 1.0662	Accuracy: 84.67%
147	Validation loss: 1.0671	Best loss: 1.0662	Accuracy: 85.33%
148	Validation loss: 1.0669	Best loss: 1.0662	Accuracy: 84.67%
149	Validation loss: 1.0670	Best loss: 1.0662	Accuracy: 84.67%
150	Validation loss: 1.0672	Best loss: 1.0662	Accuracy:

In [53]:
## Measure training time = 2.53 s ##
## train 1000 epochs without early stop
import time

n_epochs = 1000

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "Team58_HW2\Team58_HW2.ckpt")
    for var in output_layer_vars:
        var.initializer.run()
    
    ## timer start ##
    tStart = time.time()
        
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_train2, y: y_train2})
    
    ## timer end ##
    tEnd = time.time()
    
    print("training time",tEnd - tStart,"s")

INFO:tensorflow:Restoring parameters from Team58_HW2\Team58_HW2.ckpt
training time 2.527967691421509 s


In [ ]:
## 3-2 cache 5th layer -- training time: 5.30s, accuracy:82.70% ##

In [36]:
## get layer5 tensor ##
layer5_out = tf.get_default_graph().get_tensor_by_name("layer5/Elu:0")
HW3_2_saver = tf.train.Saver()
import time

## parameter setting ##
n_epochs = 1000
early_stop_max_steps = 20
steps_without_progress = 0
best_loss = 9999999


with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "Team58_HW2\Team58_HW2.ckpt")
    for var in output_layer_vars:
        var.initializer.run()
    
    ## cache layer5 output ##
    layer5_train = layer5_out.eval(feed_dict={X: X_train2, y: y_train2})
    layer5_valid = layer5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
        
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={layer5_out: layer5_train, y: y_train2})
        now_loss = sess.run(loss, feed_dict={X: X_valid2, y: y_valid2})
        now_acc = sess.run(accuracy, feed_dict={X: X_valid2, y: y_valid2})
        ## early stop implementation ##
        if now_loss < best_loss:
            HW3_2_saver.save(sess, "Team58_HW3_2\Team58_HW3_2.ckpt")
            best_loss = now_loss
            steps_without_progress = 0
        else:
            steps_without_progress += 1
            if steps_without_progress > early_stop_max_steps: ## if no progress in 20 step then early stop ##
                print("early stop")
                break
        ## print result per epoch ##
        print("{}\tValidation loss: {:.4f}\tBest loss: {:.4f}\tAccuracy: {:.2f}%".format(epoch, now_loss, best_loss, now_acc * 100))

    
    HW3_1_saver.restore(sess, "Team58_HW3_2\Team58_HW3_2.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("test accuracy",acc_test * 100,"%")

INFO:tensorflow:Restoring parameters from Team58_HW2\Team58_HW2.ckpt
0	Validation loss: 1.5964	Best loss: 1.5964	Accuracy: 26.67%
1	Validation loss: 1.5050	Best loss: 1.5050	Accuracy: 40.00%
2	Validation loss: 1.4554	Best loss: 1.4554	Accuracy: 44.00%
3	Validation loss: 1.4194	Best loss: 1.4194	Accuracy: 49.33%
4	Validation loss: 1.3876	Best loss: 1.3876	Accuracy: 50.67%
5	Validation loss: 1.3605	Best loss: 1.3605	Accuracy: 54.67%
6	Validation loss: 1.3437	Best loss: 1.3437	Accuracy: 56.00%
7	Validation loss: 1.3365	Best loss: 1.3365	Accuracy: 56.67%
8	Validation loss: 1.3361	Best loss: 1.3361	Accuracy: 57.33%
9	Validation loss: 1.3348	Best loss: 1.3348	Accuracy: 58.00%
10	Validation loss: 1.3298	Best loss: 1.3298	Accuracy: 57.33%
11	Validation loss: 1.3221	Best loss: 1.3221	Accuracy: 59.33%
12	Validation loss: 1.3148	Best loss: 1.3148	Accuracy: 59.33%
13	Validation loss: 1.3091	Best loss: 1.3091	Accuracy: 58.67%
14	Validation loss: 1.3037	Best loss: 1.3037	Accuracy: 58.00%
15	Validati

135	Validation loss: 1.0662	Best loss: 1.0662	Accuracy: 85.33%
136	Validation loss: 1.0662	Best loss: 1.0662	Accuracy: 84.67%
137	Validation loss: 1.0663	Best loss: 1.0662	Accuracy: 84.67%
138	Validation loss: 1.0667	Best loss: 1.0662	Accuracy: 84.67%
139	Validation loss: 1.0672	Best loss: 1.0662	Accuracy: 84.67%
140	Validation loss: 1.0677	Best loss: 1.0662	Accuracy: 84.67%
141	Validation loss: 1.0681	Best loss: 1.0662	Accuracy: 85.33%
142	Validation loss: 1.0684	Best loss: 1.0662	Accuracy: 85.33%
143	Validation loss: 1.0684	Best loss: 1.0662	Accuracy: 85.33%
144	Validation loss: 1.0682	Best loss: 1.0662	Accuracy: 85.33%
145	Validation loss: 1.0678	Best loss: 1.0662	Accuracy: 84.67%
146	Validation loss: 1.0674	Best loss: 1.0662	Accuracy: 84.67%
147	Validation loss: 1.0671	Best loss: 1.0662	Accuracy: 85.33%
148	Validation loss: 1.0669	Best loss: 1.0662	Accuracy: 84.67%
149	Validation loss: 1.0670	Best loss: 1.0662	Accuracy: 84.67%
150	Validation loss: 1.0672	Best loss: 1.0662	Accuracy:

In [52]:
## Measure training time = 2.23 s, faster than HW3.1 ##
## train 1000 epochs without early stop
layer5_out = tf.get_default_graph().get_tensor_by_name("layer5/Elu:0")
HW3_2_saver = tf.train.Saver()
import time

n_epochs = 1000

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "Team58_HW2\Team58_HW2.ckpt")
    for var in output_layer_vars:
        var.initializer.run()    
    ## cache layer5 output ##
    layer5_train = layer5_out.eval(feed_dict={X: X_train2, y: y_train2})
    
    ## timer begin ##
    tStart = time.time()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={layer5_out: layer5_train, y: y_train2})
    ## timer end ##
    tEnd = time.time()
    print("training time",tEnd - tStart,"s")


INFO:tensorflow:Restoring parameters from Team58_HW2\Team58_HW2.ckpt
training time 2.229403018951416 s


In [ ]:
## 3-3 4 layers instead, accuracy: 84.84% , better than HW1&HW2(82.70%) ##

In [54]:
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()
reset_graph()

restore_saver = tf.train.import_meta_graph("Team58_HW2\Team58_HW2.ckpt.meta")


X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

## get 4th layer output before training ##
layer4_out = tf.get_default_graph().get_tensor_by_name("layer4/Elu:0")
# add a new softmax layer
logits = tf.layers.dense(layer4_out, 5, kernel_initializer=he_init, name="layer4_logits")
Y_proba = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

## setting trainable variable ##
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="layer4_logits")
optimizer = tf.train.AdamOptimizer(0.01, name="Adamgram")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
HW3_3_saver = tf.train.Saver() ## creat saver ##

In [55]:
n_epochs = 1000
early_stop_max_steps = 20
steps_without_progress = 0
best_loss = 9999999

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "Team58_HW2\Team58_HW2.ckpt")
    for var in output_layer_vars:
        var.initializer.run()
        
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_train2, y: y_train2})
        now_loss = sess.run(loss, feed_dict={X: X_valid2, y: y_valid2})
        now_acc = sess.run(accuracy, feed_dict={X: X_valid2, y: y_valid2})
        ## early stop implementation ##
        if now_loss < best_loss:
            HW3_3_saver.save(sess, "Team58_HW3_3\Team58_HW3_3.ckpt")
            best_loss = now_loss
            steps_without_progress = 0
        else:
            steps_without_progress += 1
            if steps_without_progress > early_stop_max_steps:
                print("early stop")
                break
        ## print result per epoch ##
        print("{}\tValidation loss: {:.4f}\tBest loss: {:.4f}\tAccuracy: {:.2f}%".format(epoch, now_loss, best_loss, now_acc * 100))

    HW3_3_saver.restore(sess, "Team58_HW3_3\Team58_HW3_3.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("test accuracy",acc_test * 100,"%")

INFO:tensorflow:Restoring parameters from Team58_HW2\Team58_HW2.ckpt
0	Validation loss: 1.3320	Best loss: 1.3320	Accuracy: 45.33%
1	Validation loss: 1.1499	Best loss: 1.1499	Accuracy: 59.33%
2	Validation loss: 1.1539	Best loss: 1.1499	Accuracy: 54.67%
3	Validation loss: 1.1299	Best loss: 1.1299	Accuracy: 54.00%
4	Validation loss: 1.0490	Best loss: 1.0490	Accuracy: 58.67%
5	Validation loss: 0.9647	Best loss: 0.9647	Accuracy: 62.67%
6	Validation loss: 0.9276	Best loss: 0.9276	Accuracy: 68.00%
7	Validation loss: 0.9401	Best loss: 0.9276	Accuracy: 66.67%
8	Validation loss: 0.9670	Best loss: 0.9276	Accuracy: 67.33%
9	Validation loss: 0.9773	Best loss: 0.9276	Accuracy: 66.67%
10	Validation loss: 0.9611	Best loss: 0.9276	Accuracy: 68.67%
11	Validation loss: 0.9249	Best loss: 0.9249	Accuracy: 70.00%
12	Validation loss: 0.8828	Best loss: 0.8828	Accuracy: 71.33%
13	Validation loss: 0.8471	Best loss: 0.8471	Accuracy: 72.00%
14	Validation loss: 0.8226	Best loss: 0.8226	Accuracy: 72.00%
15	Validati

test accuracy 84.8385095596 %


In [ ]:
## 3-4 Bonus, accuracy: 88.95% , better than HW3(84.84%) ##

In [56]:
trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="layer1||layer2||layer4_logits")
optimizer = tf.train.AdamOptimizer(0.01, name="Adamgram")
training_op = optimizer.minimize(loss, var_list=trainable_vars)

init = tf.global_variables_initializer()
HW3_4_saver = tf.train.Saver()

In [58]:
n_epochs = 1000

early_stop_max_steps = 20
steps_without_progress = 0
best_loss = 9999999

with tf.Session() as sess:
    init.run()
    HW3_3_saver.restore(sess, "Team58_HW3_3\Team58_HW3_3.ckpt")
        
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_train2, y: y_train2})
        now_loss = sess.run(loss, feed_dict={X: X_valid2, y: y_valid2})
        now_acc = sess.run(accuracy, feed_dict={X: X_valid2, y: y_valid2})
        ## early stop implementation ##
        if now_loss < best_loss:
            HW3_4_saver.save(sess, "Team58_HW3_4\Team58_HW3_4.ckpt")
            best_loss = now_loss
            steps_without_progress = 0
        else:
            steps_without_progress += 1
            if steps_without_progress > early_stop_max_steps:
                print("early stop")
                break
        ## print result per epoch ##
        print("{}\tValidation loss: {:.4f}\tBest loss: {:.4f}\tAccuracy: {:.2f}%".format(epoch, now_loss, best_loss, now_acc * 100))

    HW3_4_saver.restore(sess, "Team58_HW3_4\Team58_HW3_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("test accuracy",acc_test * 100,"%")

INFO:tensorflow:Restoring parameters from Team58_HW3_3\Team58_HW3_3.ckpt
0	Validation loss: 2.7127	Best loss: 2.7127	Accuracy: 72.00%
1	Validation loss: 12.0427	Best loss: 2.7127	Accuracy: 50.00%
2	Validation loss: 5.4907	Best loss: 2.7127	Accuracy: 53.33%
3	Validation loss: 2.5905	Best loss: 2.5905	Accuracy: 51.33%
4	Validation loss: 2.6312	Best loss: 2.5905	Accuracy: 64.67%
5	Validation loss: 5.6698	Best loss: 2.5905	Accuracy: 56.00%
6	Validation loss: 2.2863	Best loss: 2.2863	Accuracy: 76.67%
7	Validation loss: 2.7342	Best loss: 2.2863	Accuracy: 67.33%
8	Validation loss: 2.6941	Best loss: 2.2863	Accuracy: 75.33%
9	Validation loss: 1.5286	Best loss: 1.5286	Accuracy: 74.00%
10	Validation loss: 1.0886	Best loss: 1.0886	Accuracy: 86.67%
11	Validation loss: 1.7085	Best loss: 1.0886	Accuracy: 80.00%
12	Validation loss: 0.9926	Best loss: 0.9926	Accuracy: 86.67%
13	Validation loss: 0.9248	Best loss: 0.9248	Accuracy: 88.00%
14	Validation loss: 1.3075	Best loss: 0.9248	Accuracy: 83.33%
15	Val